# Laboratorio #4 – Clasificación de Malware
## Parte 1 - Análisis de un malware

In [13]:
from pefile import PE
from typing import Dict

## Análisis estático

In [16]:
vg655_25th = PE('MALWR2/sample_vg655_25th.exe')
qwrty_dk2 = PE('MALWR2/sample_qwrty_dk2')

In [20]:
def entry_import_info(pe: PE) -> Dict:
    dlls = []
    fun_calls = []
    for entry in pe.DIRECTORY_ENTRY_IMPORT:
        dlls.append(entry.dll)
        fun_calls.extend([fun.name for fun in entry.imports])
    return {'dlls': dlls, 'fun_calls': fun_calls}
    

In [21]:
entry_import_info(vg655_25th)

{'dlls': [b'KERNEL32.dll', b'USER32.dll', b'ADVAPI32.dll', b'MSVCRT.dll'],
 'fun_calls': [b'GetFileAttributesW',
  b'GetFileSizeEx',
  b'CreateFileA',
  b'InitializeCriticalSection',
  b'DeleteCriticalSection',
  b'ReadFile',
  b'GetFileSize',
  b'WriteFile',
  b'LeaveCriticalSection',
  b'EnterCriticalSection',
  b'SetFileAttributesW',
  b'SetCurrentDirectoryW',
  b'CreateDirectoryW',
  b'GetTempPathW',
  b'GetWindowsDirectoryW',
  b'GetFileAttributesA',
  b'SizeofResource',
  b'LockResource',
  b'LoadResource',
  b'MultiByteToWideChar',
  b'Sleep',
  b'OpenMutexA',
  b'GetFullPathNameA',
  b'CopyFileA',
  b'GetModuleFileNameA',
  b'VirtualAlloc',
  b'VirtualFree',
  b'FreeLibrary',
  b'HeapAlloc',
  b'GetProcessHeap',
  b'GetModuleHandleA',
  b'SetLastError',
  b'VirtualProtect',
  b'IsBadReadPtr',
  b'HeapFree',
  b'SystemTimeToFileTime',
  b'LocalFileTimeToFileTime',
  b'CreateDirectoryA',
  b'GetStartupInfoA',
  b'SetFilePointer',
  b'SetFileTime',
  b'GetComputerNameW',
  b'GetCu

In [22]:
entry_import_info(qwrty_dk2)

{'dlls': [b'KERNEL32.DLL',
  b'MSVCRT.dll',
  b'SHELL32.dll',
  b'USER32.dll',
  b'WS2_32.dll'],
 'fun_calls': [b'LoadLibraryA',
  b'ExitProcess',
  b'GetProcAddress',
  b'VirtualProtect',
  b'atol',
  b'SHChangeNotify',
  b'LoadStringA',
  b'closesocket']}

¿Qué diferencias observa entre los ejemplos? 

En vg655_25th la cantidad de funciones utilizadas es mucho mayor que las funciones en qwrty_dk2. Algunos dlls cambian, pero no hay una gran diferencia en el numero de dlls.

¿Existe algún indicio sospechoso en la cantidad de DLLs y las APIs llamadas?

La poca cantidad de llamadas en qwrty_dk2 es sospechoso, pueden que esten empaquetadas. Ademas de algunos dlls, como `SHELL32.dll` tambien parecen sospechosos.

In [32]:
print('------- vg655_25th -------')
for section in vg655_25th.sections:
    print(section)
print('------- qwrty_dk2 -------')
for section in qwrty_dk2.sections:
    print(section)

------- vg655_25th -------
[IMAGE_SECTION_HEADER]
0x1F0      0x0   Name:                          .text
0x1F8      0x8   Misc:                          0x69B0    
0x1F8      0x8   Misc_PhysicalAddress:          0x69B0    
0x1F8      0x8   Misc_VirtualSize:              0x69B0    
0x1FC      0xC   VirtualAddress:                0x1000    
0x200      0x10  SizeOfRawData:                 0x7000    
0x204      0x14  PointerToRawData:              0x1000    
0x208      0x18  PointerToRelocations:          0x0       
0x20C      0x1C  PointerToLinenumbers:          0x0       
0x210      0x20  NumberOfRelocations:           0x0       
0x212      0x22  NumberOfLinenumbers:           0x0       
0x214      0x24  Characteristics:               0x60000020
[IMAGE_SECTION_HEADER]
0x218      0x0   Name:                          .rdata
0x220      0x8   Misc:                          0x5F70    
0x220      0x8   Misc_PhysicalAddress:          0x5F70    
0x220      0x8   Misc_VirtualSize:              0x5

¿Qué significa que algunas secciones tengan como parte de su nombre “upx”?

Significa que las llamadas estan empaquetdas. Tal como se sospechaba con qwrty_dk2, las llamadas parecen estar empaquetadas.

In [30]:
!! upx -d MALWR2/sample_vg655_25th.exe

['                       Ultimate Packer for eXecutables',
 '                          Copyright (C) 1996 - 2020',
 'UPX 3.96        Markus Oberhumer, Laszlo Molnar & John Reiser   Jan 23rd 2020',
 '',
 '        File size         Ratio      Format      Name',
 '   --------------------   ------   -----------   -----------',
 'upx: MALWR2/sample_vg655_25th.exe: NotPackedException: not packed by UPX',
 '',
 'Unpacked 0 files.']

In [31]:
!! upx -d MALWR2/qwrty_dk2.exe

['                       Ultimate Packer for eXecutables',
 '                          Copyright (C) 1996 - 2020',
 'UPX 3.96        Markus Oberhumer, Laszlo Molnar & John Reiser   Jan 23rd 2020',
 '',
 '        File size         Ratio      Format      Name',
 '   --------------------   ------   -----------   -----------',
 'upx: MALWR2/qwrty_dk2.exe: FileNotFoundException: MALWR2/qwrty_dk2.exe: No such file or directory',
 '',
 'Unpacked 0 files.']

In [33]:
vg655_25th = PE('MALWR2/sample_vg655_25th.exe')
qwrty_dk2 = PE('MALWR2/sample_qwrty_dk2')

In [34]:
print('------- vg655_25th -------')
for section in vg655_25th.sections:
    print(section)
print('------- qwrty_dk2 -------')
for section in qwrty_dk2.sections:
    print(section)

------- vg655_25th -------
[IMAGE_SECTION_HEADER]
0x1F0      0x0   Name:                          .text
0x1F8      0x8   Misc:                          0x69B0    
0x1F8      0x8   Misc_PhysicalAddress:          0x69B0    
0x1F8      0x8   Misc_VirtualSize:              0x69B0    
0x1FC      0xC   VirtualAddress:                0x1000    
0x200      0x10  SizeOfRawData:                 0x7000    
0x204      0x14  PointerToRawData:              0x1000    
0x208      0x18  PointerToRelocations:          0x0       
0x20C      0x1C  PointerToLinenumbers:          0x0       
0x210      0x20  NumberOfRelocations:           0x0       
0x212      0x22  NumberOfLinenumbers:           0x0       
0x214      0x24  Characteristics:               0x60000020
[IMAGE_SECTION_HEADER]
0x218      0x0   Name:                          .rdata
0x220      0x8   Misc:                          0x5F70    
0x220      0x8   Misc_PhysicalAddress:          0x5F70    
0x220      0x8   Misc_VirtualSize:              0x5

Según el paper “Towards Understanding Malware Behaviour by the Extraction of API Calls”,
¿en que categoría sospechosas pueden clasificarse estos ejemplos en base a algunas de las
llamadas a las APIs que realizan?

**SHA256 Hash**

In [36]:
from hashlib import sha256

In [40]:
with open("MALWR2/sample_vg655_25th.exe", "rb") as f:
    file_hash = sha256()
    while chunk := f.read(8192):
        file_hash.update(chunk)
file_hash.hexdigest()

'ed01ebfbc9eb5bbea545af4d01bf5f1071661840480439c6e5babe8e080e41aa'

Para el archivo “sample_vg655_25th.exe”, ¿cuál es el propósito de la DLL ADVAPI32.dll?

Son llamadas avanzadas de windows. Entre las funcionens que se puede utilizar es encender/apagar la computadora e iniciar procesos de windows.

https://stackoverflow.com/questions/9194638/anyone-knows-what-does-advapi-stand-for

Para el archivo “sample_vg655_25th.exe”, ¿cuál es el propósito de la API CryptReleaseContext?

Se utiliza para liberar la referencia al cryptographic service provider y al key container, esto termina la sesion con el cryptographic service provider lo que causa que todos los objetos y llaves de la sesion ya no sean validos.

https://docs.microsoft.com/en-us/windows/win32/api/wincrypt/nf-wincrypt-cryptreleasecontext

## Análisis dinámico

SRC: https://www.hybrid-analysis.com

¿Se corresponde el HASH de la plataforma con el generado?

Si, el hash generado por hybrid-analysis corresponde al generado localmente.

¿Cuáles el nombre del malware encontrado? ¿En que consiste este malware?

El nombre es `owo_im_not_ransomware_xd.exe`, es un [ransomware](https://en.wikipedia.org/wiki/Ransomware)

¿Se corresponden las sospechas con el análisis realizado en el punto 7?

Si, tiene muchos sentido que utilicen las apis de cryptografia para poder lograr encriptar la informacion del disco y que no sea factible desencriptarla al no tener llaves validas.